In [12]:
# from langchain.document_loaders import TextLoader

from langchain_community.document_loaders import TextLoader


In [13]:
# loader = TextLoader('test1.txt')

In [14]:
# 
from langchain_community.document_loaders import PDFPlumberLoader

import glob

# 配置信息
PDF_FILES_PATH = 'testpdf.pdf'
EMBEDDING_MODEL = "nomic-embed-text"
CHROMA_DB_PATH = 'chroma_db'
CHUNK_SIZE = 2000
CHUNK_OVERLAP = 100


pdf_files = glob.glob(PDF_FILES_PATH)

for file in pdf_files:
    loader = PDFPlumberLoader(file)

In [15]:
from langchain.text_splitter import CharacterTextSplitter
# from langchain.vectorstores import FAISS
from langchain_community.vectorstores import FAISS
# from langchain_community.embeddings import OllamaEmbeddings
from langchain_ollama import OllamaEmbeddings

documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
texts = text_splitter.split_documents(documents)
# embeddings = OpenAIEmbeddings()
EMBEDDING_MODEL = "nomic-embed-text"
local_embeddings = OllamaEmbeddings(model=EMBEDDING_MODEL)
db = FAISS.from_documents(texts, local_embeddings,normalize_L2=True)

In [16]:
retriever = db.as_retriever(search_type="similarity_score_threshold", search_kwargs={"score_threshold": .1})

In [17]:
docs = retriever.get_relevant_documents("允思拓公司的市场定位是什么？")

In [18]:
query="允思拓公司的市场定位是什么？"
db.similarity_search_with_relevance_scores(query)

[(Document(id='09c53262-1697-4deb-9b59-5d112120f29b', metadata={'source': 'testpdf.pdf', 'file_path': 'testpdf.pdf', 'page': 0, 'total_pages': 12, 'Author': 'wangl', 'Creator': 'Microsoft® Word 2019', 'CreationDate': "D:20250314161411+08'00'", 'ModDate': "D:20250314161411+08'00'", 'Producer': 'Microsoft® Word 2019'}, page_content='允思拓生物科技售前 AI 工程师培训资料\n第一部分：公司主营业务介绍\n1.1 公司概况\n允思拓（天津）生物科技有限公司成立于2023 年 4 月，注册资本 200 万元，总部位于天津经济技术开发区滨\n海 - 中关村科技园。公司聚焦微生物、基因组及转录组技术领域，依托生物信息学与 AI 技术整合，作为一家聚\n焦生物科技领域的创新型企业，致力于为高校、科研院所、医疗机构及农业企业提供专业化服务。\n1.2 主要产品与服务\n• 微生物：\n1、病原微生物监测预警；\n技术应用：开发新冠病毒监测预警技术、基于高通量测序的病原细菌溯源算法，实现快速病原体鉴定与传\n播链追踪；结合基因组、转录组数据，构建病原微生物进化树，辅助疫情防控与溯源决策。\n2、草图\n3、完成图\n4、扩增子\n5、宏基因组\n• 基因组：\n1、全基因组survey\n2、基因组组装、注释\n3、动植物基因组T2T分析\n4、图形泛基因组分析\n5、比较基因组分析\n6、Hic辅助染色体挂载分析\n7、全基因组甲基化分析\n8、全基因组Gwas分析\n• 转录组：\n1、原核/真核转录组分析\n2、Pacbio全长转录组分析\n3、宏转录组分析\n4、smallRNA/lncRNA分析\n1.3 技术优势\n• 核心技术：'),
  0.48889490403039415),
 (Document(id='726ecf47-9f7d-49de-bd6d-f66ee1422f60', metadata={'source': 'testp

In [19]:
from langchain_ollama import ChatOllama

model_name="deepseek-r1:8b"
llm = ChatOllama(model=model_name,
                            base_url="http://10.168.6.88:11434",temperature = 0,mirostat_tau=2.0,top_k=10,top_p=0.5
                            )

In [20]:
# Helper function for printing docs
def pretty_print_docs(docs):
    print(
        f"\n{'-' * 100}\n".join(
            [f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]
        )
    )

In [21]:
# 另一种更简单但更强大的过滤器
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainFilter

_filter = LLMChainFilter.from_llm(llm)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=_filter, base_retriever=retriever
)

compressed_docs = compression_retriever.invoke(
    "我要做细菌框架图（1G）,大概要多少钱？"
)
pretty_print_docs(compressed_docs)

# 后续结合compressed_docs和问题构建qa链得到结果

Document 1:

1、核心团队建立的基于高通量测序的病原微生物检测溯源的算法和流程，连续三年
在“联合国秘书长调查机制”能力评测中表现优异，获得相关部委表彰。疫情期间
新冠监测报告多次被中央办公厅采用，并获中央领导批示。
2、公司核心团队构建超大复杂基因组组装算法和策略。与中国科学院海洋研究所、
中国林业科学研究院、河南省农科院等单位合作完成了包括对虾、海参、扇贝、藏
野驴，芝麻和杨树等多种复杂大基因组的测序工作。
3、公司核心团队平均具有 12.5 年行业从业经验。长期从事基因组学及生物信息学
研究，专注于群体基因组进化和变异监测领域。累计开发了包括高复杂度海洋生物
组装技术、A2基因型奶牛筛选技术、病原微生物检测溯源等算法和流程。
• 研发团队：团队成员熟练掌握企业管理所需的各类专业知识，包括涉及各基因组技
术的产品开发、生产扩大、商业扩张和全球客户参与业务。主要管理人员在相关领
域深耕10年以上，积累了丰富的分析和开发经验。我们坚持紧跟科研趋势，不断创
新，为科研工作者提供最具影响力的科学成果和技术洞察。
• 知识产权：NA
1.4 市场定位
• 目标客户：高校、科研院所、医疗机构及农业企业
• 竞争优势：
1核心成员平均拥有10多年的行业经验，专注微生物、动植物基因组测序分析及算
法开发，多大10余项核心技术，包括新冠病毒监测预警、病原细菌溯源算法等；
2 价格相对优势；首先我们人工成本低，人效高，比如大规模这种公司，管理层和
人力等会占实际产生利润的人员2：3，也就是说他必须卖高价才可以维持公司正常
运转
3 我们售后服务相对优势；我们团队都是专业深耕这个领域很多年，有丰富的项目
经验，比如大规模公司他们好多都是新培养的新人去做，而且项目巨多，没有多余
时间去处理售后项目
4 项目周期相对优势；大公司会有运营对接客户，这种会延迟项目问题的解决及准
确的传达问题点，我们采用信息直接对接客户的方式，能快速解决客户的痛点及问
题
第二部分：价格体系
2.1 产品定价
样本量 <=10 <=50 <=100 数据量+1G 周期


In [22]:
query="我要做细菌框架图（1G）,大概要多少钱？"
db.similarity_search_with_relevance_scores(query)

[(Document(id='726ecf47-9f7d-49de-bd6d-f66ee1422f60', metadata={'source': 'testpdf.pdf', 'file_path': 'testpdf.pdf', 'page': 1, 'total_pages': 12, 'Author': 'wangl', 'Creator': 'Microsoft® Word 2019', 'CreationDate': "D:20250314161411+08'00'", 'ModDate': "D:20250314161411+08'00'", 'Producer': 'Microsoft® Word 2019'}, page_content='1、核心团队建立的基于高通量测序的病原微生物检测溯源的算法和流程，连续三年\n在“联合国秘书长调查机制”能力评测中表现优异，获得相关部委表彰。疫情期间\n新冠监测报告多次被中央办公厅采用，并获中央领导批示。\n2、公司核心团队构建超大复杂基因组组装算法和策略。与中国科学院海洋研究所、\n中国林业科学研究院、河南省农科院等单位合作完成了包括对虾、海参、扇贝、藏\n野驴，芝麻和杨树等多种复杂大基因组的测序工作。\n3、公司核心团队平均具有 12.5 年行业从业经验。长期从事基因组学及生物信息学\n研究，专注于群体基因组进化和变异监测领域。累计开发了包括高复杂度海洋生物\n组装技术、A2基因型奶牛筛选技术、病原微生物检测溯源等算法和流程。\n• 研发团队：团队成员熟练掌握企业管理所需的各类专业知识，包括涉及各基因组技\n术的产品开发、生产扩大、商业扩张和全球客户参与业务。主要管理人员在相关领\n域深耕10年以上，积累了丰富的分析和开发经验。我们坚持紧跟科研趋势，不断创\n新，为科研工作者提供最具影响力的科学成果和技术洞察。\n• 知识产权：NA\n1.4 市场定位\n• 目标客户：高校、科研院所、医疗机构及农业企业\n• 竞争优势：\n1核心成员平均拥有10多年的行业经验，专注微生物、动植物基因组测序分析及算\n法开发，多大10余项核心技术，包括新冠病毒监测预警、病原细菌溯源算法等；\n2 价格相对优势；首先我们人工成本低，人效高，比如大规模这种公司，管理层和\n人力等会占实际产生利润的人员2：3，也就是说他必须卖高价